In [ ]:
import geopandas
import pandas
import pathlib
import numpy
import matplotlib
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
sampling_resolution = 10

## Load in results
### Widths and slopes

In [ ]:
experiment_key = "" #_075-2thresh_65m_search_no_veg"
cache_dir = pathlib.Path(r"C:\Users\pearsonra\Documents\data\river_bathemetry\waikanae_new_lidar\caches\100000000")
width_file = cache_dir / ("final_transect_values" + experiment_key + ".geojson")
rec_main_channel_file = cache_dir / "rec_main_channel.geojson"
widths = geopandas.read_file(width_file)
main_channel = geopandas.read_file(rec_main_channel_file)

### Slopes and Flows

In [ ]:
mannings_file = pathlib.Path(r"C:\Users\pearsonra\Documents\data\river_bathemetry\mannings_n\bathy.csv.gz")
mannings = pandas.read_csv(mannings_file)

### Measured cross sections

In [ ]:
cross_section_file = pathlib.Path(r"C:/Users/pearsonra/Documents/data/river_bathemetry/sections/Waikanae/Waikanae_Cross_Sections_NIWA/2014_Survey_XS_NZTM.shp")
cross_sections = geopandas.read_file(cross_section_file)

### Local datum conversion

In [ ]:
local_datum_file = pathlib.Path(r"C:\Users\pearsonra\Documents\data\local_datums\lds-wellington-1953-to-nzvd2016-conversion-raster-GTiff\wellington-1953-to-nzvd2016-conversion-raster.tif")

In [ ]:
import rioxarray
with rioxarray.rioxarray.open_rasterio(local_datum_file, masked=True) as local_vertical_datum:
            local_vertical_datum.load()

# Translate the samples to correct reach

In [ ]:
widths['nzsegment'] = numpy.zeros(len(widths['widths']), dtype=int) * -1
for i, row in main_channel.iterrows():
    widths.loc[widths.intersects(row.geometry), ('nzsegment')] = row['nzsegment']

#### Temporary - manaully set last value

In [ ]:
widths.loc[972, ('nzsegment')]=widths['nzsegment'].iloc[971]

# Add the flows and manning's n to the widths table

In [ ]:
widths[['widths', 'valid_widths']].plot()

In [ ]:
slope_name = 'slope_mean_2.0km'
min_z_name = 'min_z_centre_unimodal_mean_2.0km' # 'min_z_centre_unimodal'
width_name = 'widths_mean_0.25km'

In [ ]:
key_parameters = widths[['nzsegment', slope_name, width_name, 'geometry', min_z_name, 'threshold']]
key_parameters = key_parameters[key_parameters['nzsegment'] != 0]

In [ ]:
key_parameters['mannings_n'] = numpy.zeros(len(key_parameters['nzsegment']), dtype=int)
key_parameters['flow'] = numpy.zeros(len(key_parameters['nzsegment']), dtype=int)
for nzsegment in key_parameters['nzsegment'].unique():
    key_parameters.loc[key_parameters['nzsegment'] == nzsegment, ('mannings_n')] = mannings[mannings['nzsegment'] == nzsegment]['n'].unique()[0]
    key_parameters.loc[key_parameters['nzsegment'] == nzsegment, ('flow')] = mannings[mannings['nzsegment'] == nzsegment]['flow'].unique()[0]

Note a section is out of order, which is why the flow goes down.

In [ ]:
key_parameters[['nzsegment']].plot()

### Take the mean of the threshold 

In [ ]:
key_parameters['rolling_threshold'] = key_parameters['threshold'].rolling(25, min_periods=1, center=True).mean()

## Calculate depth

In [ ]:
key_parameters['depth_Neal_et_al'] = (key_parameters['mannings_n'] * key_parameters['flow'] \
                                      / (numpy.sqrt(key_parameters[slope_name]) \
                                         * key_parameters[width_name])) ** (3/5) - key_parameters['rolling_threshold']

In [ ]:
a = 0.745
b = 0.305
K_0 = 6.16

In [ ]:
key_parameters['depth_Smart_et_al'] = \
    (key_parameters['flow'] / (K_0 * key_parameters[width_name] 
                               * key_parameters[slope_name] ** b)) ** (1 / (1+a)) - key_parameters['rolling_threshold']

## Plot results
Include plots of each parameter - Manning's n, flow, depth and slope - used to calculate depth

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 3))
key_parameters[['mannings_n']].plot(ax=ax, linewidth=2)
ax.set(title=f"Mannings N for {threshold}m threshold")
matplotlib.pyplot.xlabel(f"Transect number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"River Mannings M in m");

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 3))
key_parameters[['flow']].plot(ax=ax, linewidth=2)
ax.set(title=f"Flows")
matplotlib.pyplot.xlabel(f"Cross section number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"River flow in m^3/s");

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 3))
key_parameters[[width_name]].plot(ax=ax, linewidth=2)
ax.set(title=f"Widths with a rolling mean filter applied")
matplotlib.pyplot.xlabel(f"Transect number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"River width (m)");

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 3))
key_parameters[[slope_name]].plot(ax=ax, linewidth=2)
ax.set(title=f"Smoothed slope calculated from the Unimodal fit water surface levels")
matplotlib.pyplot.xlabel(f"Transect number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"River slope");

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 3))
key_parameters[['threshold','rolling_threshold']].plot(ax=ax, linewidth=2)
ax.set(title=f"Bank-full width threshold")
matplotlib.pyplot.xlabel(f"Transect number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"Bank-full width threshold");

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
key_parameters['depth_Neal_et_al'].plot(ax=ax, linewidth=2, label='River depth calculated Neal et al')
key_parameters['depth_Smart_et_al'].plot(ax=ax, linewidth=2, label='River depth calculated Rupp & Smart')
ax.set(title="Depths estimated by the two approaches (uniform flow theory approach, and hydraulic geometry)")
matplotlib.pyplot.xlabel(f"Transect number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"River depth in m");
matplotlib.pyplot.legend()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
key_parameters['depth_Neal_et_al'].plot(ax=ax, linewidth=2, label='River depth calculated Neal et al')
key_parameters['depth_Smart_et_al'].plot(ax=ax, linewidth=2, label='River depth calculated Rupp & Smart')
ax.set(title="Depths estimated by the two approaches (uniform flow theory approach, and hydraulic geometry)")
matplotlib.pyplot.xlabel(f"Transect number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"River depth in m");
matplotlib.pyplot.legend()

## Measured cross sections

Plot minimum height (i.e height of the bed)

Add the LVD conversion

In [ ]:
def get_lvd_conversion(geometry, local_vertical_datum):
    return local_vertical_datum.sel({'x': [geometry.x], 'y': [geometry.y]}, method='nearest').data[0,0,0]

cross_sections['lvd_conversion'] = cross_sections.apply(lambda row: get_lvd_conversion(row.geometry, local_vertical_datum), axis=1)
cross_sections['z_2016'] = cross_sections['Level'] - cross_sections['lvd_conversion']

Pull out the centre point for each section - minimum height - for idntification of the nearest sampled transect

In [ ]:
section_centres = {'Section': [], 'geometry': []}
for section in cross_sections['Section'].unique():
    cross_section = cross_sections[cross_sections['Section'] == section]
    section_centre = cross_section[cross_section['Level']==cross_section['Level'].min()].iloc[0]
    section_centres['Section'].append(section_centre['Section'])
    section_centres['geometry'].append(section_centre['geometry'])
section_centres = geopandas.GeoDataFrame(section_centres, crs=main_channel.crs)

Cycle through each section identifying the nearest section in the key parameters

In [ ]:
nearest_transect = numpy.zeros_like(key_parameters[slope_name], dtype=bool)
for index, row in section_centres.iterrows():
    nearest_transect[key_parameters.distance(row.geometry).argmin()] = True

nearest_key_parameters = key_parameters[nearest_transect]

## Plot the depths of the two approaches

### Compare the depth results to measured

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
(nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Neal_et_al']).reset_index(drop=True).plot(label='Bed elevation calculated Neal et al')
(nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Smart_et_al']).reset_index(drop=True).plot(label='Bed elevation calculated Rupp & Smart')
cross_sections.groupby('Section')['z_2016'].min().reset_index(drop=True).plot(label='Measured cross-section')
ax.set(title="Bed elevations estimated and measured")
matplotlib.pyplot.xlabel(f"Transect number where there are measured cross sections")
matplotlib.pyplot.ylabel(f"River bed elevation in m");
matplotlib.pyplot.legend()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
(nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Neal_et_al']).reset_index(drop=True).plot(label='Bed elevation calculated Neal et al')
(nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Smart_et_al']).reset_index(drop=True).plot(label='Bed elevation calculated Rupp & Smart')
cross_sections.groupby('Section')['z_2016'].min().reset_index(drop=True).plot(label='Measured cross-section')
ax.set(title="Bed elevations estimated and measured")
matplotlib.pyplot.xlabel(f"Transect number where there are measured cross sections")
matplotlib.pyplot.ylabel(f"River bed elevation in m");
matplotlib.pyplot.legend()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
(nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Neal_et_al']).reset_index(drop=True).plot(label='Bed elevation calculated Neal et al')
(nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Smart_et_al']).reset_index(drop=True).plot(label='Bed elevation calculated Rupp & Smart')
(nearest_key_parameters['min_z_centre_unimodal']).reset_index(drop=True).plot(linestyle='--', label='Estimated unimodal water elevation')
cross_sections.groupby('Section')['z_2016'].min().reset_index(drop=True).plot(label='Measured cross-section')
ax.set(title="Bed elevations estimated and measured")
matplotlib.pyplot.xlabel(f"Transect number where there are measured cross sections")
matplotlib.pyplot.ylabel(f"River bed elevation in m");
matplotlib.pyplot.legend()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
(cross_sections.groupby('Section')['z_2016'].min().reset_index(drop=True) 
 - ((nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Neal_et_al'])).reset_index(drop=True)).plot(label='Bed elevation error Neal et al')
(cross_sections.groupby('Section')['z_2016'].min().reset_index(drop=True) 
 - ((nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Smart_et_al'])).reset_index(drop=True)).plot(label='Bed elevation error Rupp & Smart')
ax.set(title="Bed elevations errors estimated versus measured")
matplotlib.pyplot.xlabel(f"Transect number where there are measured cross sections")
matplotlib.pyplot.ylabel(f"River bed elevation errors in m");
matplotlib.pyplot.legend()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
(cross_sections.groupby('Section')['z_2016'].min().reset_index(drop=True) 
 - ((nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Neal_et_al'])).reset_index(drop=True)).plot(label='Bed elevation error Neal et al')
(cross_sections.groupby('Section')['z_2016'].min().reset_index(drop=True) 
 - ((nearest_key_parameters[min_z_name]-nearest_key_parameters['depth_Smart_et_al'])).reset_index(drop=True)).plot(label='Bed elevation error Rupp & Smart')
ax.set(title="Bed elevations errors estimated versus measured")
matplotlib.pyplot.xlabel(f"Transect number where there are measured cross sections")
matplotlib.pyplot.ylabel(f"River bed elevation errors in m");
matplotlib.pyplot.legend()

Plot the nearest widths

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
nearest_key_parameters['rolling_threshold'].reset_index(drop=True).plot()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
key_parameters[slope_name].reset_index(drop=True).plot()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
nearest_key_parameters[slope_name].reset_index(drop=True).plot()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
nearest_key_parameters[width_name].reset_index(drop=True).plot()

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
nearest_key_parameters['depth_Neal_et_al'].reset_index(drop=True).plot(ax=ax, linewidth=2, label='River depth calculated Neal et al')
nearest_key_parameters['depth_Smart_et_al'].reset_index(drop=True).plot(ax=ax, linewidth=2, label='River depth calculated Rupp & Smart')
ax.set(title="Depths estimated by the two approaches (uniform flow theory approach, and hydraulic geometry)")
matplotlib.pyplot.xlabel(f"Transect number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"River depth in m");
matplotlib.pyplot.legend()